In [ ]:
import requests
from bs4 import BeautifulSoup
import re

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

member_list_300 = []
for index in range(1,31):
    url = f'https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex={index}'
    print(url)
    res = requests.get(url, headers=headers)
    #print(res.status_code)
    if res.ok:
        #print(res.json()['resultList'])
        member_list_10 = []
        for member in res.json()['resultList']:
            member_list_10.append(member) 
        
        #print(member_list_10)
        member_list_300.extend(member_list_10)
    else:
        print(res.status_code)
print(len(member_list_300))
print(member_list_300[:3])

In [ ]:
from pprint import pprint

pprint(member_list_300[:3])

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('===> 스크래핑 시작')
#297명의 정보를 저장하는 List
member_detail_list = []

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

for idx,member in enumerate(member_list_300,1):
    mem_sub_link = member['mLinkUrl']
    detail_url = f'https://www.assembly.go.kr{mem_sub_link}'
    res = requests.get(detail_url, headers=headers)
    print(res.ok, idx, detail_url)
    
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #1명의 정보를 저장하는 dict 선언
        member_detail_dict = {}
        
        li_tags = soup.select('ul.list li:nth-of-type(n+4)')
        #print(li_tags)
        # for li in li_tags:
        #     print(type(li.find('dt')))
        #     print(li.find('dt').text)
        
        dt_list = [li_tag.find('dt').text for li_tag in li_tags]
        #print(dt_list)

        dd_list = []        
        for li_tag in li_tags:
            dd_tag = li_tag.find('dd')
            pattern = re.compile(f'[\n\r\t]')
            dd_text = pattern.sub('',dd_tag.text.strip()).replace(" ","")
            dd_list.append(dd_text)
        #print(dd_list)
        
        member_detail_dict = dict(zip(dt_list, dd_list))

        member_detail_dict['이름'] = member['hgNm']        
        member_detail_dict['정당'] = member['polyNm']
        member_detail_dict['성별'] = member['sexGbnNm']
        member_detail_dict['선거구'] = member['origNm']
        member_detail_dict['지역비례'] = member['electGbnNm']

        sosok = member['cmitNm']
        if sosok is not None:
            idx = sosok.find(',')
            if idx > -1:            
                member_detail_dict['소속위원회'] = sosok[:idx]
            else:
                member_detail_dict['소속위원회'] = sosok
        else:        
            member_detail_dict['소속위원회'] = ''    

        # print(member_detail_dict['소속위원회'])                        
        
        member_detail_dict['당선횟수'] = member['reeleGbnNm']
        
        img_sub_url = member['mThumbImgUrl']
        img_url = urljoin(detail_url,img_sub_url)
        #print(img_url)
        member_detail_dict['이미지'] = img_url
        
        #pprint(member_detail_dict)        
                        
        #1명의 정보가 저장된 dict를 list에 추가하기
        member_detail_list.append(member_detail_dict)
    else:
        print(res.status_code)
        
print(len(member_detail_list))
print(member_detail_list[297:])
print('===> 스크래핑 끝')


In [ ]:
import pandas as pd

member_df = pd.DataFrame(columns=['이름','정당','선거구','당선횟수','지역비례','성별','소속위원회','이미지','사무실 전화','사무실 호실',\
                                  '개별 홈페이지','이메일','보좌관','선임비서관','비서관','의원실 안내'])

for member_detail in member_detail_list:
    # Row 하나 생성
    df_new_row = pd.DataFrame.from_records([member_detail])
    member_df = pd.concat([member_df, df_new_row])
    # dict <=> Series
    #series_obj = pd.Series(member_detail)
    # Series 객체를 append 
    #member_df = member_df.append(series_obj, ignore_index=True)

print(member_df.columns)
print(member_df.shape)    
member_df.head(2)

In [6]:
# index가 1부터 시작하도록 재설정
member_df.reset_index(drop=True, inplace=True)
member_df.index = member_df.index + 1
print(member_df.index)

RangeIndex(start=1, stop=1, step=1)


In [7]:
import pandas as pd

member_df = pd.read_json('data/members.json')
print(member_df.shape)

(296, 16)


In [8]:
member_df['정당'].unique()

array(['조국혁신당', '국민의힘', '더불어민주당', '무소속', '진보당', '기본소득당', '개혁신당', '사회민주당'],
      dtype=object)

In [9]:
member_df['정당'].value_counts()

정당
더불어민주당    162
국민의힘      107
조국혁신당      12
무소속         6
진보당         4
개혁신당        3
기본소득당       1
사회민주당       1
Name: count, dtype: int64

In [10]:
member_df['당선횟수'].value_counts()

당선횟수
초선    130
재선     78
3선     46
4선     24
5선     14
6선      4
Name: count, dtype: int64

In [11]:
member_df.loc[member_df['당선횟수'] == '6선',['이름','정당']]

,이름,정당
251,조경태,국민의힘
258,조정식,더불어민주당
263,주호영,국민의힘
280,추미애,더불어민주당


In [12]:
member_df.loc[member_df['당선횟수'].isin(['5선', '6선']), ['이름', '정당']]

,이름,정당
14,권성동,국민의힘
15,권영세,국민의힘
23,김기현,국민의힘
70,김태년,더불어민주당
78,나경원,국민의힘
112,박지원,더불어민주당
152,안규백,더불어민주당
166,우원식,무소속
174,윤상현,국민의힘
181,윤호중,더불어민주당


In [13]:
member_df['성별'].value_counts(normalize=True)

성별
남    0.787162
여    0.212838
Name: proportion, dtype: float64

In [ ]:
member_df['선거구'].value_counts()

In [15]:
#선거구2 컬럼을 새로 추가
#경남 창원시성산구 -> 경남
print(type(member_df['선거구']))
print(type(member_df['선거구'].str))
#Series객체를 StringMethods 객체로 변환(문자열 슬라이싱을 하기 위해서)
temp_str = member_df['선거구'].str
#2글자 슬라이싱 값을 선거구2 라는 컬럼으로 저장
member_df['선거구2'] = temp_str[:2]

member_df.loc[0:3,['선거구','선거구2']]

<class 'pandas.core.series.Series'>
<class 'pandas.core.strings.accessor.StringMethods'>


,선거구,선거구2
0,비례대표,비례
1,대구 동구군위군을,대구
2,경기 안양시만안구,경기
3,경북 구미시을,경북


In [16]:
member_df['선거구2'].value_counts()

선거구2
경기    59
서울    48
비례    46
부산    18
경남    16
경북    13
인천    13
대구    12
충남    10
전남    10
전북     9
충북     8
광주     8
강원     8
대전     7
울산     6
제주     3
세종     2
Name: count, dtype: int64

In [ ]:
# Row Counting - normalize=True 속성은 % (퍼센티지) 비율로 나타냄
member_df['선거구2'].value_counts(normalize=True)

In [ ]:
member_df['지역비례'].value_counts()

In [ ]:
member_df['지역비례'].value_counts(normalize=True)

In [ ]:
member_df['이미지'].tail()

In [ ]:
from IPython.display import Image, display

for img_url in member_df['이미지'].sample(3):
    print(img_url)
    #display(Image(url=img_url))

시각화
- Image 출력 - Image 객체, diplay() 함수 사용
- seaborn - countplot, distplot, heatmap Plot 그려보기
- maplotlib - histogram, pie Plot 그려보기
- 한글폰트 설정

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import seaborn as sns

In [ ]:
#한글폰트 설정
font_path = 'C:\\Windows\\Fonts\\malgun.ttf'
# font의 파일정보로 font name을 알아내기
font_prop = fm.FontProperties(fname=font_path).get_name()
print(font_prop)
# matplotlib의 rc(run command) 함수를 사용해서 폰트이름 설정
matplotlib.rc('font', family=font_prop)

Seaborn 의 막대 그래프
- barplot - x축과 y축을 둘 다 설정할 수 있음
- countplot - x축 이나 y축 중에서 하나만 설정할 수 있음
    - 범주형 속성을 가지는 데이터들의 히스토그램을 보여준다.
    - 종류별로 count를 보여주는 Plot이다.

In [ ]:
member_df['정당'].value_counts().index

In [ ]:
figure, (axes1,axes2,axes3) = plt.subplots(nrows=3, ncols=1)
figure.set_size_inches(18,12)

sns.countplot(data=member_df, x='정당', ax=axes1, order=member_df['정당'].value_counts().index, hue='정당')
sns.countplot(data=member_df, x='당선횟수', ax=axes2, order=member_df['당선횟수'].value_counts().index, hue='당선횟수')
sns.countplot(data=member_df, x='성별', ax=axes3, order=member_df['성별'].value_counts().index, hue='성별')

plt.show()

Matplotlib 의 Pie Plot 그려보기

In [ ]:
cdf = member_df['선거구2'].value_counts(normalize=True)
print(cdf.index)
cdf

In [ ]:
# figure size 설정
figure = plt.figure(figsize=(20,12))
print(type(figure))
# pie() 함수 - pie plot을 그려주는 함수
plt.pie(cdf, labels=cdf.index, autopct='%1.1f%%', startangle=140, shadow=True)
# pie plot을 그릴때 원의 형태를 유지하도록 하는 설정
plt.axis('equal')
plt.title('선거구 분포값')
plt.show()

Seaborn의 HistPlot (분포도)
- 히스토그램은 구간에 따른 Count를 표현하기 때문에, y는 count로 고정되어 있어 x값만 설정하면 됩니다. ( x 와 y 중에서 하나만 설정하면 됩니다.)
- hue 옵션은 값에 따라 Plot의 색을 다르게 표현할 수 있다.
- multiple = 'dodge'|'stack'

In [ ]:
figure, axes1 = plt.subplots(nrows=1, ncols=1)
figure.set_size_inches(18,12)

_ = sns.histplot(data=member_df, x='정당', ax=axes1, hue='성별', multiple='dodge',
             palette={'남': 'blue', '여': 'red'}, shrink=0.8)
plt.show()

In [ ]:
member_df.loc[member_df['정당'] == '조국혁신당',['이름','성별']]

In [ ]:
member_df.loc[member_df['정당'] == '조국혁신당',['이름','성별']].groupby('성별').count()

다중 빈도 막대 그래프 catplot
- 다중 빈도 막대그래프는 countplot() 또는 catplot() 이 있습니다.
- hue 옵션은 변량을 색상으로 구분하는 파라미터입니다.

In [ ]:
# 선거구 이름들이 빈도순으로 정렬된 리스트입니다.
s_index = member_df['선거구2'].value_counts().index
s_index

In [ ]:
# figure, axes1 = plt.subplots(nrows=1, ncols=1)
# figure.set_size_inches(24,12)

_ = sns.catplot(data=member_df, y='선거구2', hue='성별', kind='count',
             palette={'남': 'navy', '여': 'orange'},
             order=s_index)
plt.show()

히스토그램을 만드는 displot
- hue와 palette를 사용하여 색상을 지정할 수 있습니다.
- col 옵션은 성별에 따라 서브플롯을 나누어서 그려줍니다.

In [ ]:
_ = sns.displot(
    data=member_df,
    x='당선횟수',
    hue='성별',  # 성별에 따라 색상 구분
    col='성별',  # 성별에 따라 캔버스 구분
    palette={'남': 'blue', '여': 'orange'},  # 색상 지정
    kde=True,  # 커널 밀도 추정 곡선 추가 (선택 사항)
    height=5,  # 각 플롯의 높이
    aspect=1.5  # 각 플롯의 가로 세로 비율
)
plt.show()